In [1]:
from IPython.display import IFrame
import numpy as np
import pandas as pd
import json

In [2]:
def load_data(path, idx, p_success=1.0):
    filename = 'n_subbins='+str(int(idx))+f'_p_pi_suc={p_success:.5f}.csv'
    df = pd.read_csv(path + filename)
    return df

In [3]:
path_root='./data/raw/'
path_results = './data/'
path_est=('aml/', 'hmm/', 'hmm_no_exp/', 'minbins/', 'entropy_gain/')
path_test=('beryllium/', 'ideal_bk/', 'pi_success/')
path_end=('bright_R_D_0/', 'bright_R_D_0_5/', 'dark_R_D_0/', 'dark_R_D_0_5/')
n_estimators = len(path_est)
n_subbins = np.arange(5, 36)
t_detection_max = 200 # us

n_subbins_histogram = 20
#
n_subbins_avg = np.zeros((len(n_subbins)))
t_detection_avg = np.zeros((len(n_subbins)))
n_errors = np.zeros((len(n_subbins)))
n_pi_pulse_avg = np.zeros((len(n_subbins)))
histogram_nsubbin = np.zeros((n_subbins_histogram))

In [4]:
for test_idx in range(2):
    df_results = pd.DataFrame()
    df_histogram = pd.DataFrame()
    for path_end_idx in range(len(path_end)):
        for l, estimator_directory in enumerate(path_est):
            path = path_root + estimator_directory + path_test[test_idx] + path_end[path_end_idx]
            for i, j in enumerate(n_subbins):
                df = load_data(path, j)     
                n_subbins_avg_i = np.mean(df['n_subbins_0'].to_numpy())
                t_detection_avg_i = (200.0 / j) * n_subbins_avg_i

                if (path_end_idx > 1):
                    n_errors_i = len(df['prediction_0'].to_numpy()) - np.sum(df['prediction_0'].to_numpy())
                else:
                    n_errors_i = np.sum(df['prediction_0'].to_numpy())

                n_subbins_avg[i] = n_subbins_avg_i
                t_detection_avg[i] = t_detection_avg_i
                n_errors[i] = n_errors_i
                if (l == 0):
                    n_pi_pulse_avg[i] = 0
                else:
                    n_pi_pulse_avg[i] = np.mean(df['n_pi_pulses_0'].to_numpy())
                
                # include the histogram thingy
                if (j == n_subbins_histogram):
                    unique, frequency = np.unique(df['n_subbins_0'].to_numpy(), return_counts = True)
                    histogram_nsubbin[(unique-1).astype(int)] = frequency
                    df_histogram.insert(loc=0, column='hist/'+estimator_directory+path_end[path_end_idx], value=histogram_nsubbin)
                    histogram_nsubbin = np.zeros((n_subbins_histogram))
                    
            df_results.insert(loc=0, column='t_avg/'+estimator_directory+path_end[path_end_idx], value=t_detection_avg)
            df_results.insert(loc=0, column='n_errors/'+estimator_directory+path_end[path_end_idx], value=n_errors)
            df_results.insert(loc=0, column='n_pi_avg/'+estimator_directory+path_end[path_end_idx], value=n_pi_pulse_avg)
    df_histogram.insert(loc=0, column='idx', value=np.arange(1, n_subbins_histogram+1, 1))
    df_results.insert(loc=0, column='n_subbins', value=n_subbins)
    df_results.to_csv(path_results + path_test[test_idx][:-1]+'.csv')
    df_histogram.to_csv(path_results + path_test[test_idx][:-1]+'_histogram_n=' + str(n_subbins_histogram) +'_N=20000.csv')

In [5]:
test_idx = 2
p_success = np.arange(0.995, 1, 0.0005)
n_subbins = 20

n_subbins_avg = np.zeros((len(p_success)))
t_detection_avg = np.zeros((len(p_success)))
fidelity = np.zeros((len(p_success)))
n_pi_pulse_avg = np.zeros((len(p_success)))

df_results = pd.DataFrame()

for path_end_idx in range(len(path_end)):
    for l, estimator_directory in enumerate(path_est):
        if (l==0):
            continue # the AML estimator does not apply Pi-pulses...
        path = path_root + estimator_directory + path_test[test_idx] + path_end[path_end_idx]
        for i, j in enumerate(p_success):
                df = load_data(path, n_subbins, p_success=j)     
                n_subbins_avg_i = np.mean(df['n_subbins_0'].to_numpy())
                t_detection_avg_i = (200.0 / n_subbins) * n_subbins_avg_i

                if (path_end_idx > 1):
                    fidelity_i =  np.sum(df['prediction_0'].to_numpy()) / len(df['prediction_0'].to_numpy())
                else:
                    fidelity_i = (len(df['prediction_0'].to_numpy()) - np.sum(df['prediction_0'].to_numpy()))/len(df['prediction_0'].to_numpy())

                n_subbins_avg[i] = n_subbins_avg_i
                t_detection_avg[i] = t_detection_avg_i
                fidelity[i] = fidelity_i
                if l > 0:
                    n_pi_pulse_avg[i] = np.mean(df['n_pi_pulses_0'].to_numpy())
                
        df_results.insert(loc=0, column='t_avg/'+estimator_directory+path_end[path_end_idx], value=t_detection_avg)
        df_results.insert(loc=0, column='fidelity/'+estimator_directory+path_end[path_end_idx], value=fidelity)
        df_results.insert(loc=0, column='n_pi_avg/'+estimator_directory+path_end[path_end_idx], value=n_pi_pulse_avg)
                    
df_results.insert(loc=0, column='p_success', value=p_success)
df_results.to_csv(path_results + path_test[test_idx][:-1]+'.csv')

In [6]:
def load_trajectory(path, idx, p_success=1.0):
    filename = 'n_subbins='+str(int(idx))+f'_p_pi_suc={p_success:.5f}.json'
    # Opening trajectory JSON file
    f = open(path+filename)
    binned_traj = json.loads(json.load(f))
    return binned_traj

In [7]:
n_subbins = 20
n_runs = 1e6
path_est=('minbins/', 'entropy_gain/', 'hmm_no_exp/', 'hmm/')
path_traj = 'traj_analysis/'
path_end = ['bright/', 'bright_bkd/', 'dark/', 'dark_bkd/']

In [8]:
df_histogram = pd.DataFrame()
for path_end_idx in range(len(path_end)):
    for l, estimator_directory in enumerate(path_est):
        path = path_root + estimator_directory + path_traj + path_end[path_end_idx]
        binned_traj = load_trajectory(path, n_subbins)
        n_estimators = np.zeros(n_subbins)
        n_pi_pulses = np.zeros(n_subbins)
        for traj_key in binned_traj.keys():
            n_estimators_i = binned_traj[traj_key].get('probability') * n_runs
            pi_pulses_arr_i = binned_traj[traj_key].get('pi_pulse_0')
            for k, subbin_bool in enumerate(pi_pulses_arr_i):
                n_estimators[k]+=n_estimators_i
                n_pi_pulses[k]+=n_estimators_i*subbin_bool
                
        df_histogram.insert(loc=0, column='n_estimator/'+estimator_directory+path_end[path_end_idx], value=n_estimators)
        df_histogram.insert(loc=0, column='n_pi_pulse/'+estimator_directory+path_end[path_end_idx], value=n_pi_pulses)          
                
df_histogram.insert(loc=0, column='subbin_idx', value=np.arange(1, n_subbins+1, 1))
df_histogram.to_csv(path_results + 'trajectory_analysis_histogram.csv')
df_histogram              

,subbin_idx,n_pi_pulse/hmm/dark_bkd/,n_estimator/hmm/dark_bkd/,n_pi_pulse/hmm_no_exp/dark_bkd/,n_estimator/hmm_no_exp/dark_bkd/,n_pi_pulse/entropy_gain/dark_bkd/,n_estimator/entropy_gain/dark_bkd/,n_pi_pulse/minbins/dark_bkd/,n_estimator/minbins/dark_bkd/,n_pi_pulse/hmm/dark/,...,n_pi_pulse/minbins/bright_bkd/,n_estimator/minbins/bright_bkd/,n_pi_pulse/hmm/bright/,n_estimator/hmm/bright/,n_pi_pulse/hmm_no_exp/bright/,n_estimator/hmm_no_exp/bright/,n_pi_pulse/entropy_gain/bright/,n_estimator/entropy_gain/bright/,n_pi_pulse/minbins/bright/,n_estimator/minbins/bright/
0,1,975550.0,1000000.0,975290.0,1000000.0,975380.0,1000000.0,975350.0,1000000.0,1000000.0,...,280910.0,1000000.0,285310.0,1000000.0,284780.0,1000000.0,286780.0,1000000.0,285820.0,1000000.0
1,2,0.0,999730.0,0.0,1000000.0,222350.0,999990.0,0.0,1000000.0,0.0,...,0.0,863090.0,0.0,285310.0,0.0,284780.0,0.0,286780.0,0.0,285820.0
2,3,75570.0,645250.0,75390.0,866330.0,264060.0,865080.0,75000.0,866170.0,81090.0,...,267540.0,568930.0,285310.0,285310.0,284780.0,284780.0,286780.0,286780.0,285820.0,285820.0
3,4,22310.0,293550.0,22610.0,540930.0,97880.0,704160.0,22470.0,543230.0,0.0,...,7140.0,484970.0,0.0,285310.0,0.0,284780.0,0.0,286780.0,0.0,285820.0
4,5,78330.0,152620.0,77710.0,293280.0,118950.0,422650.0,77720.0,293570.0,81090.0,...,27910.0,346510.0,23510.0,81640.0,23850.0,81500.0,23050.0,81960.0,23170.0,81940.0
5,6,10570.0,124960.0,10250.0,231600.0,36950.0,410190.0,10450.0,233090.0,0.0,...,14180.0,201770.0,0.0,23510.0,0.0,23850.0,0.0,23050.0,0.0,23170.0
6,7,12830.0,73920.0,13510.0,146700.0,33230.0,208110.0,12970.0,147610.0,7050.0,...,25850.0,118280.0,23510.0,23510.0,23850.0,23850.0,23050.0,23050.0,23170.0,23170.0
7,8,6690.0,37100.0,6530.0,83590.0,16240.0,176720.0,6500.0,83430.0,0.0,...,5670.0,85390.0,0.0,23510.0,0.0,23850.0,0.0,23050.0,0.0,23170.0
8,9,8730.0,20210.0,9050.0,53090.0,13840.0,85120.0,8860.0,51660.0,7050.0,...,5750.0,52770.0,1950.0,7030.0,1720.0,6620.0,1870.0,6520.0,1800.0,6680.0
9,10,4020.0,11290.0,4100.0,35290.0,7420.0,69650.0,4300.0,33940.0,0.0,...,4520.0,32240.0,0.0,1950.0,0.0,1720.0,0.0,1870.0,0.0,1800.0


In [9]:
n_subbins = 20
n_runs = 1e6
path_est=('aml/', 'minbins/', 'entropy_gain/', 'hmm_no_exp/', 'hmm/')
path_traj = 'traj_analysis/'
path_end = ['bright/', 'bright_bkd/', 'dark/', 'dark_bkd/']

In [10]:
df_results = pd.DataFrame()
df_histogram = pd.DataFrame()
for path_end_idx in range(len(path_end)):
    for l, estimator_directory in enumerate(path_est):
        path = path_root + estimator_directory + path_traj + path_end[path_end_idx]
        df = load_data(path, n_subbins)     
        n_subbins_avg_i = np.mean(df['n_subbins_0'].to_numpy())
        t_detection_avg_i = (200.0 / n_subbins) * n_subbins_avg_i

        if (path_end_idx > 1):
            n_errors_i = len(df['prediction_0'].to_numpy()) - np.sum(df['prediction_0'].to_numpy())
        else:
            n_errors_i = np.sum(df['prediction_0'].to_numpy())

        if (l == 0):
            n_pi_pulse_avg_i = 0
        else:
            n_pi_pulse_avg_i = np.mean(df['n_pi_pulses_0'].to_numpy())

        # histogram thingy
        histogram_nsubbin = np.zeros((n_subbins_histogram))
        unique, frequency = np.unique(df['n_subbins_0'].to_numpy(), return_counts = True)
        histogram_nsubbin[(unique-1).astype(int)] = frequency
        df_histogram.insert(loc=0, column='hist/'+estimator_directory+path_end[path_end_idx], value=histogram_nsubbin)
        

        df_results.insert(loc=0, column='t_avg/'+estimator_directory+path_end[path_end_idx], value=[t_detection_avg_i])
        df_results.insert(loc=0, column='n_errors/'+estimator_directory+path_end[path_end_idx], value=[n_errors_i])
        df_results.insert(loc=0, column='n_pi_avg/'+estimator_directory+path_end[path_end_idx], value=[n_pi_pulse_avg_i])
df_histogram.insert(loc=0, column='idx', value=np.arange(1, n_subbins+1, 1))
df_results.insert(loc=0, column='n_subbins', value=[n_subbins])
df_results.to_csv(path_results + path_traj[:-1]+'.csv')
df_histogram.to_csv(path_results + 'ideal_bk_histogram_n=' + str(n_subbins) +'_N=100000.csv')

In [11]:
df_results

,n_subbins,n_pi_avg/hmm/dark_bkd/,n_errors/hmm/dark_bkd/,t_avg/hmm/dark_bkd/,n_pi_avg/hmm_no_exp/dark_bkd/,n_errors/hmm_no_exp/dark_bkd/,t_avg/hmm_no_exp/dark_bkd/,n_pi_avg/entropy_gain/dark_bkd/,n_errors/entropy_gain/dark_bkd/,t_avg/entropy_gain/dark_bkd/,...,t_avg/hmm_no_exp/bright/,n_pi_avg/entropy_gain/bright/,n_errors/entropy_gain/bright/,t_avg/entropy_gain/bright/,n_pi_avg/minbins/bright/,n_errors/minbins/bright/,t_avg/minbins/bright/,n_pi_avg/aml/bright/,n_errors/aml/bright/,t_avg/aml/bright/
0,20,1.20091,143.0,33.7122,1.20248,5.0,43.0751,1.79982,12.0,50.2441,...,20.1986,0.62376,0.0,20.2466,0.62208,0.0,20.2239,0,7.0,13.9756
